# D210 - Representation and Reporting
---

In [1]:
## Import libraries/packages
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = (18,10)
plt.rcParams['figure.max_open_warning'] = False
from datetime import datetime as dt
from scipy.stats import zscore
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import warnings
warnings.filterwarnings('ignore')

# Data Cleaning
---

In [2]:
## Define states dictionary to convert state abbreviations to names
state_dict = {
    'AL': 'Alabama',
    'AK': 'Alaska',
    'AZ': 'Arizona',
    'AR': 'Arkansas',
    'CA': 'California',
    'CO': 'Colorado',
    'CT': 'Connecticut',
    'DE': 'Delaware',
    'FL': 'Florida',
    'GA': 'Georgia',
    'HI': 'Hawaii',
    'ID': 'Idaho',
    'IL': 'Illinois',
    'IN': 'Indiana',
    'IA': 'Iowa',
    'KS': 'Kansas',
    'KY': 'Kentucky',
    'LA': 'Louisiana',
    'ME': 'Maine',
    'MD': 'Maryland',
    'MA': 'Massachusetts',
    'MI': 'Michigan',
    'MN': 'Minnesota',
    'MS': 'Mississippi',
    'MO': 'Missouri',
    'MT': 'Montana',
    'NE': 'Nebraska',
    'NV': 'Nevada',
    'NH': 'New Hampshire',
    'NJ': 'New Jersey',
    'NM': 'New Mexico',
    'NY': 'New York',
    'NC': 'North Carolina',
    'ND': 'North Dakota',
    'OH': 'Ohio',
    'OK': 'Oklahoma',
    'OR': 'Oregon',
    'PA': 'Pennsylvania',
    'RI': 'Rhode Island',
    'SC': 'South Carolina',
    'SD': 'South Dakota',
    'TN': 'Tennessee',
    'TX': 'Texas',
    'UT': 'Utah',
    'VT': 'Vermont',
    'VA': 'Virginia',
    'WA': 'Washington',
    'WV': 'West Virginia',
    'WI': 'Wisconsin',
    'WY': 'Wyoming',
    'DC': 'District of Columbia',
    'PR': 'Puerto Rico'
}

## Medical Data
---

In [3]:
## Import data
df = pd.read_csv('medical_clean.csv')

In [4]:
## Map state abbrv to name
if len(df['State'][0]) == 2:
    df['State'] = df['State'].map(state_dict)

In [5]:
## Drop extraneous columns
df1 = df.drop([
    'CaseOrder',
    'Customer_id',
    'Interaction',
    'UID',
    'City',
    'State',
    'County', 
    'Zip', 
    'Lat', 
    'Lng',
    'TimeZone', 
    'Job',
    'Item1', 
    'Item2', 
    'Item3', 
    'Item4',
    'Item5', 
    'Item6', 
    'Item7', 
    'Item8'
], axis=1)

In [6]:
## Separate string columns from numeric columns
df2 = pd.DataFrame()
for col in df1.columns:
    if df1[col].dtype == object:
        df2[col] = df1[col]
        df1 = df1.drop(col, axis=1)

In [7]:
## Remove outliers
df1 = df1.drop(zscore(df1)[abs(zscore(df1)) > 3].dropna(thresh=1).index.values)

In [8]:
## Remove outlier observations from string columns
df2 = df2.loc[df1.index]

In [9]:
## Encode binary variables
labels = {'Yes': 1, 'No':0}
for col in df2.columns:
    if 'Yes' in df2[col].unique():
        df1[col] = df2[col].map(labels)
        df2 = df2.drop(col, axis=1)

In [10]:
## Keep relevant rows and columns
df[df1.columns] = df1
df = df[df.columns[4:]]
df = df.loc[df1.index]
df = df.reset_index(drop=True)

In [11]:
## Get dummy variables
df1 = pd.concat([df1, pd.get_dummies(df2)], axis=1)

In [12]:
## Reset index
df1 = df1.reset_index(drop=True)

In [13]:
## Split training and testing data
X, y = df1.drop('ReAdmis', axis=1), df1[['ReAdmis']]
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [14]:
## Train logistic regression model
clf = LogisticRegression(random_state=10).fit(X_train, y_train)

In [15]:
## Get the mean accuracy score
print(f'Accuracy: {round(clf.score(X_test, y_test)*100, 2)}%')

Accuracy: 94.21%


In [16]:
## Get the expected readmission rate
df['ExpectedReAdmis'] = clf.predict(X)

In [17]:
## Create admission column
df['Admissions'] = 1

In [18]:
## Review clean data
df

,City,State,County,Zip,Lat,Lng,Population,Area,TimeZone,Job,...,Item1,Item2,Item3,Item4,Item5,Item6,Item7,Item8,ExpectedReAdmis,Admissions
0,Eva,Alabama,Morgan,35621,34.34960,-86.72508,2951.0,Suburban,America/Chicago,"Psychologist, sport and exercise",...,3,3,2,2,4,3,3,4,0,1
1,Marianna,Florida,Jackson,32446,30.84513,-85.22907,11303.0,Urban,America/Chicago,Community development worker,...,3,4,3,4,4,4,3,3,0,1
2,Sioux Falls,South Dakota,Minnehaha,57110,43.54321,-96.63772,17125.0,Suburban,America/Chicago,Chief Executive Officer,...,2,4,4,4,3,4,3,3,0,1
3,New Richland,Minnesota,Waseca,56072,43.89744,-93.51479,2162.0,Suburban,America/Chicago,Early years teacher,...,3,5,5,3,4,5,5,5,0,1
4,West Point,Virginia,King William,23181,37.59894,-76.88958,5287.0,Rural,America/New_York,Health promotion specialist,...,2,1,3,3,5,3,4,3,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9326,Norlina,North Carolina,Warren,27563,36.42886,-78.23716,4762.0,Urban,America/New_York,"Programmer, multimedia",...,3,2,2,3,4,3,4,2,1,1
9327,Milmay,New Jersey,Atlantic,8340,39.43609,-74.87302,1251.0,Urban,America/New_York,"Restaurant manager, fast food",...,3,3,4,2,5,3,4,4,1,1
9328,Southside,Tennessee,Montgomery,37171,36.36655,-87.29988,532.0,Rural,America/Chicago,"Psychologist, occupational",...,3,3,3,4,4,2,3,2,1,1
9329,Quinn,South Dakota,Pennington,57775,44.10354,-102.01590,271.0,Rural,America/Denver,Outdoor activities/education manager,...,5,5,3,4,4,3,4,3,1,1


In [19]:
## Store clean data
df.to_csv('medical_data_clean.csv', index=False)

## Readmission Data
---

In [20]:
## Import data
df = pd.read_csv('FY_2023_Hospital_Readmissions_Reduction_Program_Hospital.csv')

In [21]:
## Review data shape and types
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18990 entries, 0 to 18989
Data columns (total 12 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Facility Name               18990 non-null  object 
 1   Facility ID                 18990 non-null  int64  
 2   State                       18990 non-null  object 
 3   Measure Name                18990 non-null  object 
 4   Number of Discharges        9339 non-null   float64
 5   Footnote                    6071 non-null   float64
 6   Excess Readmission Ratio    12919 non-null  float64
 7   Predicted Readmission Rate  12919 non-null  float64
 8   Expected Readmission Rate   12919 non-null  float64
 9   Number of Readmissions      12919 non-null  object 
 10  Start Date                  18990 non-null  object 
 11  End Date                    18990 non-null  object 
dtypes: float64(5), int64(1), object(6)
memory usage: 1.7+ MB


In [22]:
## Review df
df.head(3)

,Facility Name,Facility ID,State,Measure Name,Number of Discharges,Footnote,Excess Readmission Ratio,Predicted Readmission Rate,Expected Readmission Rate,Number of Readmissions,Start Date,End Date
0,SOUTHEAST HEALTH MEDICAL CENTER,10001,AL,READM-30-HIP-KNEE-HRRP,NaN,NaN,1.0301,4.5819,4.4481,Too Few to Report,07/01/2018,06/30/2021
1,SOUTHEAST HEALTH MEDICAL CENTER,10001,AL,READM-30-COPD-HRRP,202.0,NaN,0.9903,19.1885,19.3765,38,07/01/2018,06/30/2021
2,SOUTHEAST HEALTH MEDICAL CENTER,10001,AL,READM-30-AMI-HRRP,319.0,NaN,0.9958,14.4888,14.5498,46,07/01/2018,06/30/2021


In [23]:
## Drop extraneous columns
df.drop([
    'Measure Name', 'Footnote', 
    'Excess Readmission Ratio', 
    'Predicted Readmission Rate',
], axis=1, inplace=True)

In [24]:
## Drop missing values
df.dropna(inplace=True)

In [25]:
## Drop duplicate values
df.drop(df[df.duplicated()].index, inplace=True)

In [26]:
## Verify no duplicates left
df.duplicated().sum()

0

In [27]:
## Reset df index
df.reset_index(drop=True, inplace=True)

In [28]:
## Change numeric values to integers
df['Number of Discharges'] = df['Number of Discharges'].astype(int)
df['Number of Readmissions'] = df['Number of Readmissions'].astype(int)

In [29]:
## Extract start and end year
start = dt.strptime(df['Start Date'][0], '%m/%d/%Y').year
end = dt.strptime(df['End Date'][0], '%m/%d/%Y').year

In [30]:
## Calculate average annual values
df['Number of Discharges'] = round(df['Number of Discharges'] / (end - start)).astype(int)
df['Number of Readmissions'] = round(df['Number of Readmissions'] / (end - start)).astype(int)

In [31]:
## Drop date columns
df.drop(['Start Date', 'End Date'], axis=1, inplace=True)

In [32]:
## Rename columns
df.columns = [
    'Hospital Name', 
    'Hospital ID', 
    'State', 
    'Avg Annual Admissions', 
    'Avg Expected Readmission Rate', 
    'Avg Annual Readmissions'
]

In [33]:
## Map state abbvr to name
df['State'] = df['State'].map(state_dict)

In [34]:
## Store clean data
df.to_csv('readm_clean.csv', index=False)